# Module 4 Final Project Submission
## Time Series Analysis

Student name: Fernando Aguilar Islas<br>
Student pace: Part-Time<br>
Scheduled project review date/time:<br>
Instructor name: Brandon Lewis<br>
Blog post URL:<br>
Video Walkthrough URL:<br>

## 1. Introduction
For this project I will be performing time series analysis on housing historical data obtained from Zillow. Real Estate, Inc. is looking for the 5 best zipcodes in the US to begin developing its newest multi-family residential complex. Real Estate, Inc. is looking for busy and dense metropolitan areas with an average house prices around the median.

## 2. Methodology

First, I will select the relevant zipcodes by using EDA. I'm only interested in zipcodes with a size rank in the top 20% size rank. Then, I will select zipcodes with an average house price between 1 decile above and 1.5 deciles below teh median. I will calculate the periodic returns and perform time series analysis to forecast next year's return.

In regards to the tme series analysis I will be using the Box–Jenkins method:

1. Identification. Use the data and all related information to help select a sub-class of model that may best summarize the data.
2. Estimation. Use the data to train the parameters of the model (i.e. the coefficients).
3. Diagnostic Checking. Evaluate the fitted model in the context of the available data and check for areas where the model may be improved. source: https://machinelearningmastery.com/gentle-introduction-box-jenkins-method-time-series-forecasting/

In [1]:
#Import the necessary libraries for EDA, visualization and time series modelling.
#Data visualization and manipulation
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.simplefilter('ignore')
plt.style.use('ggplot')
%matplotlib inline
#Time series analysis tools.
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf


In [2]:
#Helper Functions
#Obtain dates form the data. Source: Flatiron School
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

#Convert the data into long format. Source: Flatiron School
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

## 3. Exploratory Data Analysis (Zipcode Selection)

In [3]:
#Instantiate pandas dataframe with housing data.
df_zillow = pd.read_csv('zillow_data.csv')
print(df_zillow.info(),'\n')
print(f'Unique zipcodes: {df_zillow.RegionName.nunique()}')
df_zillow.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB
None 

Unique zipcodes: 14723


,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


From this initial EDA, I can see that the data is provided in wide-format. Further down the project I will transform the data into a long-format. There are 14,723 unique zipcodes in the data, hence the next step will be to select the zipcodes that fit Real Estate, Inc. target market requirements.

In [4]:
#Get zipcodes with a size rank in the top 20% (highly urbanized zipcodes).
print(df_zillow.SizeRank.describe(),'\n')
#Calculate the 20% cutoff value.
sr_20 = df_zillow.SizeRank.quantile(q=0.20)
print(f'Size Rank 20% cutoff value: {sr_20}')
#Get data frame with selected zipcodes. Keep values and zipcodes only.
zc_top20= df_zillow[df_zillow['SizeRank']<sr_20].drop(['RegionID','City','State','Metro','CountyName','SizeRank'],axis=1)
print(f'Amount of zipcodes: {len(zc_top20)}')

count    14723.000000
mean      7362.000000
std       4250.308342
min          1.000000
25%       3681.500000
50%       7362.000000
75%      11042.500000
max      14723.000000
Name: SizeRank, dtype: float64 

Size Rank 20% cutoff value: 2945.4
Amount of zipcodes: 2945


The number of zipcodes to consider has been narrowed down from a total of 14,723 to only a fifth of that, 2,945. Now, I will look into zipcodes with an average value of between 1 decile above the median and 1.5 deciles below the median. I will calculate the average value using data form last year to represent the prices the company is likely to encounter currenlty.

In [5]:
zc_top20['yr_avg']=zc_top20.iloc[:,-12:].mean(skipna=True, axis=1)
#Get zipcodes with an average value 1 decile above the median and 1.5 deciles below.
print(zc_top20['yr_avg'].describe(),'\n')
#Calculate the 60% cutoff value (1 decile above).
q_60 = zc_top20['yr_avg'].quantile(q=0.60)
print(f'Average Value 60% cutoff value: {round(q_60,2)}')
#Calculate the 35% cutoff value (1.5 deciles below).
q_35 = zc_top20['yr_avg'].quantile(q=0.35)
print(f'Average Value 35% cutoff value: {round(q_35,2)}')
#Get data frame with selected zipcodes.
zc_pref= zc_top20[(zc_top20['yr_avg']<q_60) & (zc_top20['yr_avg']>q_35)]
print(f'Amount of zipcodes: {len(zc_pref)}')

count    2.945000e+03
mean     3.730666e+05
std      5.836511e+05
min      3.632500e+04
25%      1.691333e+05
50%      2.484083e+05
75%      3.978333e+05
max      1.858138e+07
Name: yr_avg, dtype: float64 

Average Value 60% cutoff value: 290211.67
Average Value 35% cutoff value: 198641.67
Amount of zipcodes: 736


The number of zipcodes to consider is now 736. This takes into account the urbanization and home value preferences of the real estate company. It is time to look into the investement perspective, hence, I will look at the historical return on investment,standard deviation and calculate the coefficient of variance to take into account the risk profile of the firm.

In [6]:
#Calculate historical return on investment
zc_pref['ROI']= (zc_pref['2018-04']/zc_pref['1996-04'])-1
#Calculate standard deviation of monthly values
zc_pref['std']=zc_pref.loc[:,'1996-04':'2018-04'].std(skipna=True, axis=1)
#Calculate historical mean value
zc_pref['mean']=zc_pref.loc[:,'1996-04':'2018-04'].mean(skipna=True, axis=1)
#Calculate coefficient of variance
zc_pref['CV']=zc_pref['std']/zc_pref['mean']
#Show calculated values
zc_pref[['RegionName','std','mean','ROI','CV']].head()

,RegionName,std,mean,ROI,CV
11,32162,53805.394161,183692.830189,1.493069,0.292910
14,37013,19771.938500,139191.698113,0.885231,0.142048
17,37211,36496.608464,147387.924528,1.698672,0.247623
18,78660,24894.592870,168193.584906,0.748020,0.148012
22,77573,29647.359699,183261.509434,0.848656,0.161776


In [7]:
#Descriptive statistics of coefficients of variance.
print(zc_pref.CV.describe())
#Define upper limit of CV according to risk profile.
upper_cv = zc_pref.CV.quantile(.6)
print(f'\nCV upper limit: {upper_cv}')
#Get the 5 zipcodes with highest ROIs within the firms risk profile.
zc_best5 = zc_pref[zc_pref['CV']<upper_cv].sort_values('ROI',axis=0,ascending=False)[:5]
zc_best5[['RegionName','ROI','CV']]

count    736.000000
mean       0.238700
std        0.080020
min        0.056119
25%        0.171094
50%        0.226459
75%        0.291877
max        0.453303
Name: CV, dtype: float64

CV upper limit: 0.25459687610566145


,RegionName,ROI,CV
1784,70808,2.258519,0.251359
1877,29461,2.061224,0.249053
2213,3820,1.988142,0.248366
1375,52722,1.948396,0.244641
2931,70809,1.930894,0.239323


In [8]:
#Get Location Names
zipcodes = list(zc_best5.RegionName.values)
for i in zipcodes:
    city = df_zillow[df_zillow['RegionName']==i].City.values[0]
    state = df_zillow[df_zillow['RegionName']==i].State.values[0]
    print(f'Zipcode : {i} \nLocation: {city}, {state}\n')

Zipcode : 70808 
Location: Baton Rouge, LA

Zipcode : 29461 
Location: Moncks Corner, SC

Zipcode : 3820 
Location: Dover, NH

Zipcode : 52722 
Location: Bettendorf, IA

Zipcode : 70809 
Location: Baton Rouge, LA



I haveve successfully identified the best 5 zipcodes according to Real Estate, Incs. target market income preferences by choosing zip codes with current avearge home values around the median. Also, the zip codes were narrowed down by size rank, with a preference for urban areas. At the end, another criteria was the highest return on investment given the risk profile of the developer. Now, its time to perform time series analysis on this zip codes to develop a model that makes accuarate return forecasts.

## 4 Time Series Analysis

In [9]:
#Change data to long-format
zc20_melt = melt_data(zc_pref).set_index('time')
zc20_melt.head()

ValueError: ('Unknown string format:', 'yr_avg')

In [ ]:
dfs_zc = []
for zc in zc20_melt.RegionName.unique():
    df = zc20_melt[zc20_melt['RegionName']==zc].asfreq('MS')
    dfs_zc.append(df)
dfs_zc[0].head()

In [ ]:
for zc in range(5):
    dfs_zc[zc]['return']=np.nan*len(dfs_zc[zc])
    for i in range(len(dfs_zc[zc])-1):
        dfs_zc[zc]['return'][i+1]= (dfs_zc[zc].value.iloc[i+1] / dfs_zc[zc].value.iloc[i]) - 1

In [ ]:
results = []
for i in range(5):
    zc = dfs_zc[i].RegionName[0]
    df = dfs_zc[i].dropna()
    roi = (df['return'] + 1).prod() -1
    gmean = pow((df['return'] + 1).prod(),(1/len(mock)))-1
    results.append((zc,roi,gmean))
results = pd.DataFrame(results,columns = ['Zipcode', 'ROI', 'MontlyMean'])

In [ ]:
results.sort_values('MontlyMean')

In [ ]:
results.sort_values('ROI')

In [ ]:
gmean = pow((mock['return'] + 1).prod(),(1/len(mock)))-1
gmean*100*12

In [ ]:
# Create figure
fig, (ax1, ax2) = plt.subplots(2,1,figsize=(12,8))
# Plot seasonal ACF
plot_acf(ts_zc20.diff(periods=12).dropna(), ax=ax1)
# Plot seasonal PACF
plot_pacf(ts_zc20.diff(periods=12).dropna(), ax=ax2)
plt.show()


In [ ]:
autocorrelation_plot(ts_zc20.diff().dropna())

In [ ]:
aloo = zc20_melt.groupby('time').mean()
aloo[aloo['value']==aloo['2009':'2014'].min()[1]].index

In [ ]:
mock_df=df.copy()
mock_df['std']=mock_df.iloc[:,7:].std(axis=1)
mock_df['avg']=mock_df.iloc[:,7:-1].mean(axis=1)
mock_df['ROI']=(mock_df.iloc[:,-3] - mock_df.iloc[:,7])/mock_df.iloc[:,7] 
mock_df['cv']=(mock_df['std']/mock_df['avg'])*100

In [ ]:
mock_df[['RegionName','City','State','ROI','cv','std']].sort_values('ROI',ascending=False)[:10]

In [ ]:
mock_df.iloc[:,-5]

In [ ]:
florida = df[df['State']=='FL'].drop(['RegionID','City','State','Metro','CountyName','SizeRank'],axis=1)
florida.head()

In [ ]:
melt_mock = df.copy()

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName','RegionID','City','State','Metro','CountyName','SizeRank'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

In [ ]:
melt_mock = melt_data(df)
zc60657 = melt_mock[melt_mock['RegionName']==60657].copy()

In [ ]:
zc60657.set_index('time',inplace=True)
zc60657.index

In [ ]:
zc60657

In [ ]:
for i in range(len(zc60657)-1):
    zc60657['yr_return'][i+1]=((zc60657['value'][i+1]-zc60657['value'][i])/zc60657['value'][i])*100
zc60657.tail()

In [ ]:
zc60657['yr_return']=0*len(zc60657)

In [ ]:
zc60657['yr_return']